In [ ]:
#process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

### Number of tracks depending on constraints
iterates over the million playlist dataset and outputs number songs tha appear more than: 1,2,3,4 times

In [ ]:
import sys
import json
import re
import collections
import os

total_playlists = 0
total_tracks = 0
track_histogram = collections.Counter()

quick = False
max_files_for_quick_processing = 1

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            process_info(mpd_slice['info'])
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    show_summary()


def show_summary():
    print()
    
    t1 = len(list(filter(lambda (x,y): y>1, track_histogram.items())))
    t2 = len(list(filter(lambda (x,y): y>2, track_histogram.items())))
    t3 = len(list(filter(lambda (x,y): y>3, track_histogram.items())))
    t4 = len(list(filter(lambda (x,y): y>4, track_histogram.items())))
    t5 = len(list(filter(lambda (x,y): y>5, track_histogram.items())))
    unique = len(track_histogram)
    
    print ("number of playlists", total_playlists)
    print ("number of tracks", total_tracks)
    print ("number of unique tracks", unique)
    print ("number of unique tracks appearing more than 1 time:" + str(t1) + "that is "+ str(t1/(unique*1.0)+"%" )
    print ("number of unique tracks appearing more than 2 times:"+ str(t2) + "that is "+ str(t2/(unique*1.0)+"%" )
    print ("number of unique tracks appearing more than 3 times:"+ str(t3) + "that is "+ str(t3/(unique*1.)+"%"  )
    print ("number of unique tracks appearing more than 4 times:"+ str(t4) + "that is "+ str(t4/(unique*1.0)+"%"  )
    print ("number of unique tracks appearing more than 5 times:"+ str(t5) + "that is "+ str(t5/(unique*1.0)+"%" )


def normalize_name(name):
    name = name.lower()
    name = re.sub(r"[.,\/#!$%\^\*;:{}=\_`~()@]", ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name


def process_playlist(playlist):
    global total_tracks, total_playlists
    total_playlists += 1
    for track in playlist['tracks']:
        total_tracks += 1
        track_histogram[track['track_uri']] += 1


def process_info(_):
    pass


if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = False
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

In [ ]:
"""
>1 100%
>2 44%
>3 30%
>4 20%
>5 ...
"""

### Dictionary file track_uri --> ID creation
now let's make a dictionary of all the tracks appearing mre than x times and uri-id

In [3]:
"""
in this notebook we will create a csv file with all the tracks_uris and its new associated unique numeric ID
only with tracks appearing more than num = 4 times 
data format: id track_uri
usage: process_mpd(path)
"""

import pandas as pd
import numpy as np
import sys
import json
import re
import collections
import os
import datetime
import csv

total_tracks = 0
track_histogram = collections.Counter()
num = 4

quick = True
max_files_for_quick_processing = 1

def process_mpd(path):
    count = 0
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice['playlists']:
                process_playlist(playlist)
            count += 1

            if quick and count > max_files_for_quick_processing:
                break

    show_summary()
    write_file()

def write_file():
    j = 0
    with open('id_trackuri_filter.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(["id","track_uri"])
        for i in range (len(track_histogram)):
            #only considers the tracks appearing at least 5 times  
            if track_histogram.values()[i]>4:               
                #we write only the last part of the track_uri, deleting "spotify:track:"
                #spamwriter.writerow([i,tracks.pop()[14:]])
                spamwriter.writerow([j,track_histogram.items()[i][0][14:]])
                j+=1
        

def show_summary():
    print("number of tracks", total_tracks)
    print("number of unique tracks", len(track_histogram))


def process_playlist(playlist):
    global total_tracks
    for track in playlist['tracks']:
        total_tracks += 1
        track_histogram[track['track_uri']] += 1

if __name__ == '__main__':
    path = sys.argv[1]
    if len(sys.argv) > 2 and sys.argv[2] == '--quick':
        quick = True
    #process_mpd(path)
    process_mpd(os.getcwd() + os.sep  + os.pardir +  "/MPD")

number of tracks 134125
number of unique tracks 57884


TypeError: 'dict_values' object does not support indexing

In [5]:
track_histogram.values()

TypeError: 'dict_values' object does not support indexing